<a href="https://colab.research.google.com/github/NavitB/IRProject/blob/main/run_frontend_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

In [3]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [4]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

for details.

--2022-01-02 17:27:09--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.237.133.81, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  59.2MB/s    in 0.2s    

2022-01-02 17:27:09 (59.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip


In [5]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
# !./ngrok authtoken YOUR_NGROK_TOKEN_HERE

In [6]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

In [7]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 100 not upgraded.
--2022-01-02 17:27:38--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 52.85.146.3, 52.85.146.12, 52.85.146.59, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|52.85.146.3|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



In [9]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [10]:
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 100 not upgraded.


In [11]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [12]:
pandasDf = pd.read_csv("/content/newdata.csv")
pandasDf = pandasDf.drop(columns = "Unnamed: 0")
df_data = spark.createDataFrame(pandasDf)
# df_data_rdd = df_data.rdd
# print(df_data_rdd.take(1))
id_text = df_data.select('text','id')
id_title = df_data.select('title','id')
id_anchor = df_data.select('id', 'anchor_text')
id_text = id_text.rdd
id_title = id_title.rdd
id_anchor = id_anchor.rdd
# print(id_anchor.take(2))
# print(id_text.take(1))
# print(id_title.take(5))

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  tuples = []
  tf_dict = Counter(tokens)
  res = []
  [res.append(x) for x in tokens if x not in res]
  for t in res:
    tuples.append((t,(id,tf_dict[t])))
  return tuples

In [14]:
word_counts_text = id_text.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_title = id_title.flatMap(lambda x: word_count(x[0], x[1]))
print(word_counts_text.take(40))
#anchor?

[('time', (4055486, 8)), ('walker', (4055486, 6)), ('1982', (4055486, 3)), ('science', (4055486, 8)), ('fiction', (4055486, 4)), ('horror', (4055486, 4)), ('film', (4055486, 11)), ('directed', (4055486, 1)), ('tom', (4055486, 3)), ('kennedy', (4055486, 1)), ('since', (4055486, 1)), ('release', (4055486, 2)), ('received', (4055486, 1)), ('negative', (4055486, 2)), ('reviews', (4055486, 1)), ('critics', (4055486, 1)), ('title', (4055486, 2)), ('another', (4055486, 5)), ('planet', (4055486, 6)), ('featured', (4055486, 2)), ('mystery', (4055486, 5)), ('theater', (4055486, 5)), ('3000', (4055486, 5)), ('episode', (4055486, 7)), ('405', (4055486, 1)), ('first', (4055486, 2)), ('aired', (4055486, 2)), ('july', (4055486, 2)), ('1992', (4055486, 2)), ('shown', (4055486, 1)), ('metv', (4055486, 1)), ('show', (4055486, 1)), ('svengoolie', (4055486, 1)), ('june', (4055486, 1)), ('2021', (4055486, 1)), ('plot', (4055486, 1)), ('california', (4055486, 4)), ('university', (4055486, 4)), ('sciences', 

In [15]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl)

In [26]:
#create postings and sort by doc_id
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
#anchor?

#filter out rare words - appearing in less than 10 docs
postings_filtered_text = postings_text.filter(lambda x: len(x[1])>10)
postings_filtered_title = postings_title.filter(lambda x: len(x[1])>10)
#anchor?

In [27]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  rdd2 = postings.map(lambda x: (x[0], len(x[1]))) 
  return rdd2

In [45]:
w2df_text = calculate_df(postings_filtered_text)
w2df_title = calculate_df(postings_filtered_title)
# print(w2df_title.take(20))

w2df_text_dict = w2df_text.collectAsMap()
w2df_title_dict = w2df_title.collectAsMap()
print(w2df_title_dict['python'])


46


In [29]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings,base_dir):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  rdd2 = postings.map(lambda x: (token2bucket_id(x[0]),x)).groupByKey()
  rdd3 = rdd2.map(lambda x: InvertedIndex.write_a_posting_list(x,base_dir))
  return rdd3


In [30]:
#rdd for postings locations
!mkdir text_pl title_pl 
postings_locs_text = partition_postings_and_write(postings_filtered_text,'text_pl').collect()
postings_locs_title = partition_postings_and_write(postings_filtered_title,'title_pl').collect()


mkdir: cannot create directory ‘text_pl’: File exists
mkdir: cannot create directory ‘title_pl’: File exists


In [38]:
#create super set for allpotings locs
super_posting_locs_text = defaultdict(list)
for posting_loc in postings_locs_text:
  for k, v in posting_loc.items():
    super_posting_locs_text[k].extend(v)

super_posting_locs_title = defaultdict(list)
for posting_loc in postings_locs_title:
  for k, v in posting_loc.items():
    super_posting_locs_title[k].extend(v)


#anchor?

In [32]:
inverted_text = InvertedIndex()
inverted_title = InvertedIndex()

inverted_text.posting_locs = super_posting_locs_text
inverted_title.posting_locs = super_posting_locs_title

inverted_text.df = w2df_text_dict
inverted_title.df = w2df_title_dict

inverted_text.write_index('.', 'index_text')
inverted_title.write_index('.', 'index_title')

In [33]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [63]:
pl_title = read_posting_list(inverted_title, 'masks')
pl_text = read_posting_list(inverted_text, 'masks')
df_title = len(pl_title)
df_text = len(pl_text)

# lst = [23862, 23329, 53672527, 21356332, 4920126, 5250192, 819149, 46448252, 83036, 88595, 18942, 696712, 2032271, 1984246, 5204237, 645111, 18384111, 3673376, 25061839, 271890, 226402, 2380213, 1179348, 15586616, 50278739, 19701, 3596573, 4225907, 19160, 1235986, 6908561, 3594951, 18805500, 5087621, 25049240, 2432299, 381782, 9603954, 390263, 317752, 38007831, 2564605, 13370873, 2403126, 17402165, 23678545, 7837468, 23954341, 11505904, 196698, 34292335, 52042, 2247376, 15858, 11322015, 13062829, 38833779, 7800160, 24193668, 440018, 54351136, 28887886, 19620, 23045823, 43003632, 746577, 1211612, 8305253, 14985517, 30796675, 51800, 964717, 6146589, 13024, 11583987, 57294217, 27471338, 5479462]
lst = set([63631542, 821829, 633458, 63567907, 14208718, 7912947, 159546, 67759831, 7439323, 64192678, 40909056, 5322079, 63164437, 31728348, 12772, 1402997, 1156703, 1840653, 568164, 1814699, 2945076, 2565663, 442333, 33567480, 468313, 19718702, 987724, 903187, 1485962, 5003908, 4201044, 28086000, 28844729, 1265651, 4659608, 64966775, 55960921, 8970797, 4301719, 35892659, 2248622, 6939163, 48561519, 261396, 34298473, 12263290, 44258772, 25219375, 149426, 6558203, 46567337, 46784964, 6458321, 46576311, 18823362, 48315099, 56440599, 57159776, 74910, 67891964, 1702593, 560306, 1015304, 705756, 7479199, 57772096, 58946599, 1210300, 15716827, 34336876, 9040490, 439102, 264104, 11061915, 39774839, 34735506, 42174581, 66859257, 3293969, 16315, 23166476, 42812440, 66935753, 51990351, 36619752, 59863622, 38404, 64027932])
print("len")
print(len(lst))
pl_id_title = [x[0] for x in pl_title]
print(pl_id_title)
pl_id_text = [x[0] for x in pl_text]
print(pl_id_text)

eq_title_text = len([x for x in pl_id_text if x in pl_id_title])
print(eq_title_text)

eq_text_lst = len([x for x in pl_id_text if x in lst])
print(eq_text_lst)

eq_title_lst = len([x for x in pl_id_title if x in lst])
print(eq_title_lst)




len
88
[987724, 4201044, 6939163, 7912947, 9040490, 18823362, 25219375, 35892659, 39774839, 42812440, 46576311, 48561519, 55960921, 57159776, 63631542, 64192678]
[12772, 16315, 38404, 74910, 149426, 159546, 180251, 193714, 264104, 348111, 348499, 439102, 468313, 568164, 633458, 821829, 879962, 903187, 909036, 987724, 1015304, 1156703, 1210300, 1249019, 1265651, 1485962, 1655005, 1702593, 1814699, 1840653, 1890951, 2248622, 2565663, 2945076, 3057990, 3293969, 4201044, 4355398, 4659608, 5003908, 5322079, 6458321, 6558203, 6939163, 7439323, 7479199, 7912947, 8304348, 8970797, 9040490, 11061915, 14208718, 15716827, 18823362, 25219375, 28086000, 28844729, 31728348, 32047012, 33567480, 34298473, 34336876, 34735506, 35892659, 36619752, 38153033, 39774839, 40909056, 42812440, 44258772, 46567337, 46576311, 46784964, 48315099, 48561519, 55960921, 56440599, 57159776, 58708106, 58946599, 63164437, 63567907, 63631542, 64027932, 64192678, 64639133, 64966775, 65217527, 66293350, 66859257, 66935753, 6

# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
import search_frontend as se

In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(se)

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [ ]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))